<a href="https://colab.research.google.com/github/DiogoNeves/tool_usage_example/blob/main/Tool_Usage_Guessing_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic

In [ ]:
from typing import Any
from collections import namedtuple

In [ ]:
import anthropic
from google.colab import userdata

ANTHROPIC_KEY = userdata.get("ANTHROPIC_API_KEY")
MODEL = "claude-3-haiku-20240307"

client = anthropic.Anthropic(api_key=ANTHROPIC_KEY)

In [ ]:
SYSTEM_MESSAGE = ("You are a game host guiding the user. The user has to"
                  " guess a secret word. Each correct guess adds 100"
                  " points to the user, each failed guess removes 10 points"
                  " from the user. The user can request their current score"
                  " which does not count as a failed guess! You have to use"
                  " the tools provided to get the secret word. You have to tell"
                  " the user if they guessed correctly and their current score,"
                  " once they place their guess. The user will play multiple"
                  " games and can chit-chat between games.")

TOOLS = [
    {
        "name": "get_secret",
        "description": ("Get the secret word the user has to guess. The output"
                        " is a single word."),
        "input_schema": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    },
    {
        "name": "add_to_score",
        "description": ("Add `points` to the total score and returns the current"
                        " total score after adding the `points`. You want to"
                        " provide the new `points` added, not the total score."
                        " `points` can be negative, meaning the player lost"
                        " `points`."),
        "input_schema": {
            "type": "object",
            "properties": {
                "points": {
                    "type": "integer",
                    "description": ("The number of points to add to the total"
                                    " score. Can be negative to remove points.")
                }
            },
            "required": ["points"],
        },
    },
    {
        "name": "get_score",
        "description": "Get the current total score.",
        "input_schema": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }
]

In [ ]:
StateType = namedtuple("StateType", "current_score")

def get_secret(internal_state: StateType) -> tuple[str, StateType]:
    return "train", internal_state


def add_to_score(points: int, internal_state: StateType) -> tuple[int, StateType]:
    new_state = StateType(current_score=internal_state.current_score + points)
    return str(internal_state.current_score), new_state


def get_score(internal_state: StateType) -> tuple[int, StateType]:
    return str(internal_state.current_score), internal_state

In [ ]:
FUNCTION_MAP = {
    "get_secret": get_secret,
    "add_to_score": add_to_score,
    "get_score": get_score
}


def run_tool(tool_block: Any, internal_state: StateType) -> tuple[dict, StateType]:
    """Run the right tool to the tool_block."""
    print(f"Called out to {tool_block.name}")

    tool_to_run = FUNCTION_MAP.get(tool_block.name, None)
    assert tool_to_run is not None

    tool_id = tool_block.id
    result, new_internal_state = tool_to_run(**tool_block.input,
                                             internal_state=internal_state)
    return {
        "type": "tool_result",
        "tool_use_id": tool_id,
        "content": str(result)
    }, new_internal_state


def game_host(user_prompt: str | list, conversation: list,
              internal_state: StateType) -> tuple[str, list, StateType]:

    messages = conversation + [{"role": "user", "content": user_prompt}]

    message = client.beta.tools.messages.create(
        model=MODEL,
        max_tokens=1000,
        temperature=0.0,
        system=SYSTEM_MESSAGE,
        tools=TOOLS,
        messages=messages
    )

    assistant_turn = {"role": "assistant", "content": message.content}
    new_conversation = messages + [assistant_turn]

    tool_blocks = [block for block in message.content
                   if block.type == "tool_use"]

    if not tool_blocks:
        return "".join([block.text for block in message.content]), \
            new_conversation, internal_state

    tool_results = []
    for block in tool_blocks:
        tool_result, internal_state = run_tool(block, internal_state)
        tool_results.append(tool_result)

    return game_host(tool_results, new_conversation, internal_state)

In [ ]:
game_state = StateType(current_score=1000)

In [ ]:
response, conversation, game_state = game_host("let's start the game", [], game_state)
print("Conversation: ", conversation)
print("Response: ", response)
print("State: ", game_state)

Conversation:  [{'role': 'user', 'content': "let's start the game"}, {'role': 'assistant', 'content': [TextBlock(text="Okay, let's start the game! I've got the secret word ready. Go ahead and make your first guess.", type='text')]}]
Response:  Okay, let's start the game! I've got the secret word ready. Go ahead and make your first guess.
State:  StateType(current_score=1000)


In [ ]:
response, conversation, game_state = game_host("train", conversation, game_state)
print("Conversation: ", conversation)
print("Response: ", response)
print("State: ", game_state)

Called out to get_secret
Called out to add_to_score
Conversation:  [{'role': 'user', 'content': "let's start the game"}, {'role': 'assistant', 'content': [TextBlock(text="Okay, let's start the game! I've got the secret word ready. Go ahead and make your first guess.", type='text')]}, {'role': 'user', 'content': 'train'}, {'role': 'assistant', 'content': [ToolUseBlock(id='toolu_015HZGRZDp1K4rHCP2X4S1b3', input={}, name='get_secret', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_015HZGRZDp1K4rHCP2X4S1b3', 'content': 'train'}]}, {'role': 'assistant', 'content': [TextBlock(text="Excellent guess! You got the secret word right. That's 100 points for you.", type='text'), ToolUseBlock(id='toolu_0184Tex7BwravezAmDfEEX6D', input={'points': 100}, name='add_to_score', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0184Tex7BwravezAmDfEEX6D', 'content': '1000'}]}, {'role': 'assistant', 'content': [TextBl

In [ ]:
response, conversation, game_state = game_host("yes", conversation, game_state)
print("Conversation: ", conversation)
print("Response: ", response)
print("State: ", game_state)

Conversation:  [{'role': 'user', 'content': "let's start the game"}, {'role': 'assistant', 'content': [TextBlock(text="Okay, let's start the game! I've got the secret word ready. Go ahead and make your first guess.", type='text')]}, {'role': 'user', 'content': 'train'}, {'role': 'assistant', 'content': [ToolUseBlock(id='toolu_015HZGRZDp1K4rHCP2X4S1b3', input={}, name='get_secret', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_015HZGRZDp1K4rHCP2X4S1b3', 'content': 'train'}]}, {'role': 'assistant', 'content': [TextBlock(text="Excellent guess! You got the secret word right. That's 100 points for you.", type='text'), ToolUseBlock(id='toolu_0184Tex7BwravezAmDfEEX6D', input={'points': 100}, name='add_to_score', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0184Tex7BwravezAmDfEEX6D', 'content': '1000'}]}, {'role': 'assistant', 'content': [TextBlock(text='Your current score is 1000 points. Ready f

In [ ]:
response, conversation, game_state = game_host("what's my score again?", conversation, game_state)
print("Conversation: ", conversation)
print("Response: ", response)
print("State: ", game_state)

Called out to get_score
Conversation:  [{'role': 'user', 'content': "let's start the game"}, {'role': 'assistant', 'content': [TextBlock(text="Okay, let's start the game! I've got the secret word ready. Go ahead and make your first guess.", type='text')]}, {'role': 'user', 'content': 'train'}, {'role': 'assistant', 'content': [ToolUseBlock(id='toolu_015HZGRZDp1K4rHCP2X4S1b3', input={}, name='get_secret', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_015HZGRZDp1K4rHCP2X4S1b3', 'content': 'train'}]}, {'role': 'assistant', 'content': [TextBlock(text="Excellent guess! You got the secret word right. That's 100 points for you.", type='text'), ToolUseBlock(id='toolu_0184Tex7BwravezAmDfEEX6D', input={'points': 100}, name='add_to_score', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0184Tex7BwravezAmDfEEX6D', 'content': '1000'}]}, {'role': 'assistant', 'content': [TextBlock(text='Your current score

In [ ]:
response, conversation, game_state = game_host("car", conversation, game_state)
print("Conversation: ", conversation)
print("Response: ", response)
print("State: ", game_state)

Called out to add_to_score
Conversation:  [{'role': 'user', 'content': "let's start the game"}, {'role': 'assistant', 'content': [TextBlock(text="Okay, let's start the game! I've got the secret word ready. Go ahead and make your first guess.", type='text')]}, {'role': 'user', 'content': 'train'}, {'role': 'assistant', 'content': [ToolUseBlock(id='toolu_015HZGRZDp1K4rHCP2X4S1b3', input={}, name='get_secret', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_015HZGRZDp1K4rHCP2X4S1b3', 'content': 'train'}]}, {'role': 'assistant', 'content': [TextBlock(text="Excellent guess! You got the secret word right. That's 100 points for you.", type='text'), ToolUseBlock(id='toolu_0184Tex7BwravezAmDfEEX6D', input={'points': 100}, name='add_to_score', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0184Tex7BwravezAmDfEEX6D', 'content': '1000'}]}, {'role': 'assistant', 'content': [TextBlock(text='Your current sc